<a href="https://colab.research.google.com/github/Catatau52/Carteira_Quant/blob/main/Etapa_03_Avaliar_premios_de_risco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mplcyberpunk

In [ ]:
import pandas as pd
import numpy as np
import mplcyberpunk
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import os
import matplotlib
matplotlib.rcParams.update({'font.size': 9})
import datetime
plt.style.use("cyberpunk")


In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=0f8252062c9aab1c8770ef3723ac1fc9b9b06f14d96760aa71dc2565c936ba6f
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/Libs')

from report_pdf_fatores import MakePDF

Mounted at /content/drive


In [ ]:

class MakeResultsPremium:

    def __init__(self, dicionario_fatores, data_final_analise, caminho_imagens, nome_arquivo = 'premios_de_risco.pdf',
                 caminho_premios_de_risco = '.'):


        self.dicionario_fatores = dicionario_fatores
        self.lista_nome_fatores = []
        self.liquidez = []
        self.caminho_premios_de_risco = caminho_premios_de_risco

        for key, item in dicionario_fatores.items():

            self.lista_nome_fatores.append(key)
            self.liquidez.append(item)


        self.data_final_analise = (datetime.datetime.strptime(data_final_analise, '%Y-%m-%d')).date()
        self.caminho_imagens = caminho_imagens
        self.nome_arquivo = nome_arquivo
        os.chdir(caminho_imagens)

    def puxando_dados(self):

        lista_dfs = []
        data_inicial = []

        for i, nome_premio in enumerate(self.lista_nome_fatores):

            #df = pd.read_parquet(f'{self.caminho_premios_de_risco}/{nome_premio}_{self.liquidez[i]}.parquet')
            df = pd.read_parquet(f'{self.caminho_premios_de_risco}/{nome_premio}.parquet')
            df['data'] = pd.to_datetime(df['data']).dt.date

            lista_dfs.append(df)
            data_inicial.append(min(df['data']))

        self.premios_de_risco = pd.concat(lista_dfs)
        data_inicial = max(data_inicial)

        self.premios_de_risco = self.premios_de_risco[(self.premios_de_risco['data'] >= data_inicial) &
                                                      (self.premios_de_risco['data'] <= self.data_final_analise)]

        #self.premios_de_risco = self.premios_de_risco.assign(premio_fator =
        #                                                    (1 + self.premios_de_risco['primeiro_quartil'])/(1 + self.premios_de_risco['quarto_quartil']))
        self.premios_de_risco = self.premios_de_risco.assign(premio_fator =
                                                             (1 + self.premios_de_risco['primeiro_quartil'])/(1 + self.premios_de_risco['universo']))
        self.premios_de_risco = self.premios_de_risco.assign(premio_fator_2 =
                                                             (1 + self.premios_de_risco['primeiro_quartil'])/(1 + self.premios_de_risco['quarto_quartil']))


        self.premios_de_risco['primeiro_quartil'] = 1 + self.premios_de_risco['primeiro_quartil']
        self.premios_de_risco['segundo_quartil'] = 1 + self.premios_de_risco['segundo_quartil']
        self.premios_de_risco['terceiro_quartil'] = 1 + self.premios_de_risco['terceiro_quartil']
        self.premios_de_risco['quarto_quartil'] = 1 + self.premios_de_risco['quarto_quartil']
        #self.premios_de_risco['quinto_quartil'] = 1 + self.premios_de_risco['quinto_quartil']
        self.premios_de_risco['universo'] = 1 + self.premios_de_risco['universo']

    def retorno_quartis(self):

        df_primeiro_quartis = pd.DataFrame(index = self.premios_de_risco['data'].unique())
        df_premios_de_risco = pd.DataFrame(index = self.premios_de_risco['data'].unique())

        for i, nome_premio in enumerate(self.lista_nome_fatores):

            fator = self.premios_de_risco[(self.premios_de_risco['nome_premio'] == nome_premio) &
                                            (self.premios_de_risco['liquidez'] == self.liquidez[i])]

            acum_primeiro_quartil = (fator['primeiro_quartil'].cumprod() - 1).iloc[-1]
            acum_segundo_quartil = (fator['segundo_quartil'].cumprod() - 1).iloc[-1]
            acum_terceiro_quartil = (fator['terceiro_quartil'].cumprod() - 1).iloc[-1]
            acum_quarto_quartil = (fator['quarto_quartil'].cumprod() - 1).iloc[-1]
            #acum_quinto_quartil = (fator['quinto_quartil'].cumprod() - 1).iloc[-1]

            fig, ax = plt.subplots(figsize = (4.75, 4))

            ax.bar(0, acum_primeiro_quartil)
            ax.bar(1, acum_segundo_quartil)
            ax.bar(2, acum_terceiro_quartil)
            ax.bar(3, acum_quarto_quartil)
            #ax.bar(4, acum_quinto_quartil)

            ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

            plt.xticks([0, 1, 2, 3], ['1º Quartil', '2º Quartil', '3º Quartil', '4º Quartil'])
            #plt.xticks([0, 1, 2, 3, 4], ['1º Quartil', '2º Quartil', '3º Quartil', '4º Quartil','5º Quartil'])

            plt.title(nome_premio)

            plt.savefig(f'{self.caminho_imagens}/barras_quartis_{nome_premio}_{self.liquidez[i]}')

            plt.close()

            fig, ax = plt.subplots(figsize = (4.75, 4))

            ax.plot(fator['data'].values, (fator['primeiro_quartil'].cumprod() - 1), label = '1º Quartil')
            ax.plot(fator['data'].values, (fator['segundo_quartil'].cumprod() - 1), label = '2º Quartil')
            ax.plot(fator['data'].values, (fator['terceiro_quartil'].cumprod() - 1), label = '3º Quartil')
            ax.plot(fator['data'].values, (fator['quarto_quartil'].cumprod() - 1), label = '4º Quartil')
            #ax.plot(fator['data'].values, (fator['quinto_quartil'].cumprod() - 1), label = '5º Quartil')
            ax.plot(fator['data'].values, (fator['universo'].cumprod() - 1), label = 'Universo')

            plt.legend()

            ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

            plt.title(nome_premio)

            plt.savefig(f'{self.caminho_imagens}/linha_quartis_{nome_premio}_{self.liquidez[i]}')

            plt.close()

            #grafico de prêmio do fator

            fig, ax = plt.subplots(figsize = (4.75, 4))

            #ax.plot(fator['data'].values, (fator['premio_fator'].cumprod() - 1))
            ax.plot(fator['data'].values, (fator['premio_fator_2'].cumprod() - 1))

            ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

            plt.title(nome_premio + " 1º Quartil minus 4º quartil")

            plt.savefig(f'{self.caminho_imagens}/premio_de_risco_{nome_premio}_{self.liquidez[i]}')

            plt.close()

            #janela movel

            serie_movel = pd.Series(data = fator['premio_fator'].rolling(12).apply(np.prod, raw = True) - 1)

            serie_movel.index = fator['data'].values

            serie_movel = serie_movel.dropna()

            fig, ax = plt.subplots(figsize = (4.75, 4))

            ax.plot(serie_movel.index, serie_movel.values)

            ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

            plt.title(nome_premio + " Janela móvel 12M")

            plt.savefig(f'{self.caminho_imagens}/movel_12m_premio_de_risco_{nome_premio}_{self.liquidez[i]}')

            plt.close()

            df_primeiro_quartis.loc[:, f"{nome_premio}"] = (fator['primeiro_quartil'].cumprod() - 1).values
            df_premios_de_risco.loc[:, f"{nome_premio}"] = (fator['premio_fator'].cumprod() - 1).values

        matplotlib.rcParams.update({'font.size': 11})

        fig, ax = plt.subplots(figsize = (9, 4.5))

        for coluna in df_primeiro_quartis.columns:

            ax.plot(df_primeiro_quartis.index, df_primeiro_quartis[coluna].values, label = f'{coluna}')

        plt.legend()

        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

        plt.title('Comparação entre 1º quartil')

        plt.savefig(f'{self.caminho_imagens}/comparando_1Q')

        plt.close()

        fig, ax = plt.subplots(figsize = (9, 4.5))

        for coluna in df_premios_de_risco.columns:

            ax.plot(df_premios_de_risco.index, df_premios_de_risco[coluna].values, label = f'{coluna}')

        plt.legend()

        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))

        plt.title('Comparação entre prêmios de risco')

        plt.savefig(f'{self.caminho_imagens}/comparando_premios')

        plt.close()

        self.matriz_correl = df_premios_de_risco.corr()


    def fazer_pdf(self):

        MakePDF(fatores = self.lista_nome_fatores, liquidez = self.liquidez, matriz_correl = self.matriz_correl,
                caminho_imagens=self.caminho_imagens, nome_arquivo=self.nome_arquivo)





if __name__ == "__main__":

    dicionario_fatores = {
                          #'ROE': 1000000,
                          #'ROIC': 1000000,
                          #'EBIT_EV': 1000000,
                          #'EV_EBIT': 1000000,
                          #'L_P': 1000000,
                          #'P_L': 1000000,
                          #'mm_7_40': 1000000, #
                          #'ValorDeMercado_Cres': 1000000,
                          #'ValorDeMercado_Decres': 1000000,
                          #'indicador_pl_db': 1000000,
                          #'indicador_vpa': 1000000,
                          #'ebit_divliq': 1000000, #
                          #'LPA': 1000000,
                          # 'indicador_lucro_ev' : 1000000,
                          # 'indicador_receita_ev' : 1000000, #
                          # 'indicador_lucro12m_ev' : 1000000,
                          # 'indicador_receita12m_ev' : 1000000,
                          #'indicador_EV_Div' : 1000000, #
                          #'My_EV_ebit' : 1000000, #
                          #'My_ebit_EV' : 1000000,
                          #'indicador_ebit_Receita' : 1000000,
                          #'indicador_lucro_Receita' : 1000000
                          #'p_vp' : 1000000, #
                          #'p_sr' : 1000000, #
                          #'indicador_p_ebit' : 1000000,
                          #'indicador_Peg_ratio' : 1000000
                          # 'momento_1_meses' : 1000000,
                          # 'momento_3_meses' : 1000000,
                          # 'momento_6_meses' : 1000000,
                          # 'momento_12_meses' : 1000000,
                          # 'mm_9_21' : 1000000,
                          # 'mm_13_26' : 1000000,
                          # 'mm_13_48' : 1000000,
                          # 'mm_14_40' : 1000000,
                          'mm_20_100' : 1000000,
                          'mm_50_200' : 1000000,
                          'mm_20_80' : 1000000,
                          'mm_20_50' : 1000000,

                           }

    premios = MakeResultsPremium(data_final_analise="2023-07-31", dicionario_fatores=dicionario_fatores,
                                 caminho_imagens = r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/imagens',
                                 nome_arquivo = r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/backtest/avaliando_todos_premios.pdf',
                                 caminho_premios_de_risco=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br')



    premios.puxando_dados()
    premios.retorno_quartis()
    premios.fazer_pdf()




In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/indicador_Peg_ratio.parquet')
df.head(3)

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/Premio_risco_ValorDeMercado_1000000.parquet')
df.head(3)

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/Premio_risco_indicador_pl_ta_1000000.parquet')
df.head(3)